In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from darts import TimeSeries

from aare.constants import LOC_BERN, LOC_THUN, TIME, TEMP
from aare.remote_existenz_store import RemoteExistenzStore
from aare.utils import *

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
store = RemoteExistenzStore()

In [ ]:
freq = "1h"
ANYTIME = "0"  # to be used as period start when querying influx. starting at 0 just returns all the data.

In [ ]:
df = store.query_hydro(ANYTIME, LOC_BERN, agg_freq=freq)
o_df = df.copy()
df

In [ ]:
# restore original df for iterative development, can re-run if necessary
df = o_df.copy()

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
# there shouldn't be any NaNs because we set agg_create_empty to False (default)
df.isna().value_counts()

In [ ]:
# at max one is allowed, at the end of the series
assert (df.set_index(TIME).resample(freq).count() > 1).sum().item() <= 1, "Has more than one data points within one time-step according to frequency"

In [ ]:
# resample to add nan points where data is missing. also removes the trailing data point if 18:00 and 18:55 for example.
# doing this manually gives a bit more control and avoid having to send this data over the air from the influx server.
df = df.set_index(TIME).resample(freq).first().reset_index(TIME)

In [ ]:
df.shape

In [ ]:
df.isna().value_counts()

In [ ]:
px.line(df, TIME, TEMP)

In [ ]:
px.scatter(df, TIME, TEMP).update_traces(marker={'size': 1})

We can clearly see that there is a lot of data missing from 2003 to 2009 although the existing data seems plausible and follows the seasonality and trend. It just has a lower frequency? Since px.line only connects points if there is no NaN in between, this period almost looks non-existent with a line plot, but in a scatter plot, it looks mostly fine.

We can also clearly see some outliers that extend below the 0° border, which doesn't make sense. This is also reflected in the dataset summary, where the minimum is -9.5°.

Manually creeping up on the Y-axis to see what the lowest likely valid temperature is, it seems that 2.5° would be a good cutoff to remove outliers on the low end.

Similarly, the maximum temperature of 25° is above the highest recorded temperature according to other sources and the only occurrence of it is clearly an outlier (2009-07-06), so threshold that away as well.

In [ ]:
df.describe().T

Manual visual analysis shows that there is a period from Jan 2003 to Mar 2003 with very strange data. This data should be excluded.
You could assume that this is the start of some measurement difficulties that are only remedied in 2009, so unless we find that it's not enough data, it might be best to exclude everything from Jan 2003 up to Jul 2009, where everything seems to be in order again.

EDIT: Given that the data in those 6 years seems to be trivially salvageable, let's just run with it and interpolate where necessary.

In [ ]:
df.loc[between(df, "2003-01-28", "2003-03-03"), TEMP] = np.nan
# this works if the time is the index
# df.loc["2003-01-28":"2003-03-03", TEMP] = np.nan

In [ ]:
low_cutoff = 2.5
high_cutoff = 25

In [ ]:
print("x <= 0°C:", np.count_nonzero(df[TEMP] <= 0))
print(f"0 < x <= {low_cutoff}°C:", np.count_nonzero((df[TEMP] > 0) & (df[TEMP] <= low_cutoff)))
print(f"x >= {high_cutoff}°C:", np.count_nonzero(df[TEMP] >= high_cutoff))

In [ ]:
outlier_mask = (df[TEMP] <= low_cutoff) | (df[TEMP] >= high_cutoff)
px.scatter(df, x=TIME, y=TEMP, color=outlier_mask).update_traces(marker={'size': 3})

In [ ]:
# eliminate all data points outside valid bound
df.loc[outlier_mask, TEMP] = np.nan

In [ ]:
df.shape

In [ ]:
df.describe().T

In [ ]:
px.scatter(df, TIME, TEMP).update_traces(marker={'size': 2})

TODO I think I got it all wrong, I just need to interpolate a lot more from 2003 to 2009 because it has a lot of 2 hour frequency instead of 1 hour. Maybe also investigate what the actual source (influx) provides BEFORE aggregation to an hour.

TODO also another type of outlier is stray data points that have no data to their left and right for more than a few hours (see Mar 2002 below).

After cleanup, it appears that the data from June 10th 2009 onwards is best. The big gap with very little data from 2003 to 2009 is probably unusable.
The data before that (Jun 2001 - Jun 2003) seems mostly usable but has some large gaps as well.

It might be easier to discard just this data as it's less than 2 years worth and could have non-negligible differences in measurement methodology, distribution, etc. compared to the new data from 6 years later. Comparing (Jun 2001 - Jun 2003) to (Jun 2009 - Jun 2011) (see below) doesn't raise any warning flags that the data prior to 2009 would be invalid. However, discarding it also loses about 18 months of data and leaves us with 15 years (180 months) of continuous data; that's a 9% loss.

If experimentation shows that more data would be helpful, efforts to recover & clean the data prior to 2009 can be made. To start experimentation and modelling, the 15+ years after should be enough.

In [ ]:
px.scatter(df.loc[between(df, "2001-06-01", "2003-06-01")], TIME, TEMP).update_traces(marker={'size': 2})

In [ ]:
px.scatter(df.loc[between(df, "2009-06-01", "2011-06-01")], TIME, TEMP).update_traces(marker={'size': 2})

Visually inspecting the data we can still see

* Random downward spikes of unrealistic magnitude
* Occasional gaps

Apart from those, the data seems very clean already.

In [ ]:
df.describe().T

In [ ]:
df[TEMP].isna().value_counts()

In [ ]:
df["temp_diff_to_prev"] = df[TEMP].diff().abs()
df["temp_diff_to_next"] = df[TEMP].diff(-1).abs()

In [ ]:
outlier_quantile = 0.999
outlier_diff = max(df["temp_diff_to_prev"].quantile(outlier_quantile), df["temp_diff_to_next"].quantile(outlier_quantile))
outlier_diff

In [ ]:
df[(df["temp_diff_to_prev"] > outlier_diff) | (df["temp_diff_to_next"] > outlier_diff)]

In [ ]:
df[(df["temp_diff_to_prev"] > outlier_diff) & (df["temp_diff_to_next"] > outlier_diff)]

One variant of outlier is at the start and end of measurement, so [NaN, outlier, normal measurement, ...] or reverse. This is a common pattern in industry sensor data measurements at least from my experience. \
A slight variation of this first variant is the case when the prev or next measurement is exactly 0 instead of NaN. \
Another is a random drop or spike so [normal, outlier, normal]. These have both diffs above threshold.

All of these variants appear in the data.

Ps. another analysis like that might be necessary after interpolating the gaps.

In [ ]:
# variant 1a
df.loc[(df["temp_diff_to_prev"].isna() | (df["temp_diff_to_prev"] == 0)) & (df["temp_diff_to_next"] > outlier_diff), TEMP] = np.nan
# variant 1b
df.loc[(df["temp_diff_to_next"].isna() | (df["temp_diff_to_next"] == 0)) & (df["temp_diff_to_prev"] > outlier_diff), TEMP] = np.nan
# variant 2
df.loc[(df["temp_diff_to_prev"] > outlier_diff) & (df["temp_diff_to_next"] > outlier_diff), TEMP] = np.nan

In [ ]:
# update diffs because outliers have now been removed (set to NaN)
df["temp_diff_to_prev"] = df[TEMP].diff().abs()
df["temp_diff_to_next"] = df[TEMP].diff(-1).abs()

In [ ]:
# all of these look legit, although they certainly fall outside the norm
df[(df["temp_diff_to_prev"] > outlier_diff) | (df["temp_diff_to_next"] > outlier_diff)]

In [ ]:
px.scatter(df, TIME, TEMP).update_traces(marker={'size': 2})

Moving to Darts now helps with gap analysis.

In [ ]:
ts = to_ts(df)
ts

In [ ]:
ts.gaps()

In [ ]:
ts.gaps().value_counts("gap_size")

Most gaps are size 1. This is also fairly visible in the data, for example at the end of 2013. The data doesn't appear to be wrong, just more sparse than it should be.

In [ ]:
df[TEMP].isna().value_counts()

In [ ]:
df[TEMP].interpolate(limit=1).isna().value_counts()

In [ ]:
df["temp_filled"] = df[TEMP].interpolate(limit=1)
df["was_filled"] = (df[TEMP].isna() & ~df["temp_filled"].isna()).astype(np.uint8)

In [ ]:
px.scatter(df, x=TIME, y="temp_filled", color="was_filled").update_traces(marker={'size': 3})

In [ ]:
df["temp_filled0"] = df[TEMP]
df["was_filled"] = 0
df["was_filled"] = df["was_filled"].astype(np.uint8)
# multiple steps of interpolation with just limit 1 to see where the larger gaps are
for i in range(1, 25):
    p, c = f"temp_filled{i-1}", f"temp_filled{i}"
    df[c] = df[p].interpolate(limit=1, limit_area="inside")
    df["was_filled"] += i * (df[p].isna() & ~df[c].isna()).astype(np.uint8)
    df["temp_filled"] = df[c]

In [ ]:
px.scatter(df, x=TIME, y="temp_filled", color="was_filled").update_traces(marker={'size': 3})

TODO Found some wild outlier on 2009-07-06, maybe move start date a bit later (e.g. 09.07.) or manually remove that outlier (can threshold to 25 for example).

In [ ]:
df["temp_filled0"] = df[TEMP]
df["was_filled"] = 0
df["was_filled"] = df["was_filled"].astype(np.uint8)
# multiple steps of interpolation with just limit 1 to see where the larger gaps are
for i in range(1, 25):
    p, c = f"temp_filled{i-1}", f"temp_filled{i}"
    df[c] = df[p].interpolate(method="cubic", limit=1, limit_area="inside")
    df["was_filled"] += i * (df[p].isna() & ~df[c].isna()).astype(np.uint8)
    df["temp_filled"] = df[c]

In [ ]:
px.scatter(df, x=TIME, y="temp_filled", color="was_filled").update_traces(marker={'size': 3})

In [ ]:
df_i = fill_with_hard_limit(df, limit=5, columns=[TEMP], add_was_filled=True)
px.scatter(df_i, x=TIME, y=TEMP, color=TEMP+"_filled").update_traces(marker={'size': 3})

TODO Outlier 2012-08-04, must be removed manually like the other one in the beginning.

Manually scanning the plot shows that linear interpolation is fine for gaps up to 5 hours. This could be tuned and done much more rigorously, but it's not really necessary here and likely only yields marginal benefits.

In [ ]:
df["temp_filled"].isna().value_counts()

In [ ]:
df["was_filled"].value_counts()

In [ ]:
to_ts(df_i[[TIME, TEMP]]).gaps().sort_values("gap_size", ascending=False)

Incorporate this interpolation into the real dataset and keep track of which portions were filled how.

In [ ]:
df[TEMP] = df_i[TEMP]
df["filled"] = df_i[TEMP+"_filled"].map({ False: "none", True: "linear" })

In [ ]:
px.scatter(df, x=TIME, y=TEMP, color="filled").update_traces(marker={'size': 3})

Given the remaining few gaps, a maximum gap size of 25 or 30 seems appropriate to be filled with a slightly more complex method. \
The results are promising because the trend is retained well in March 2022 for those shorter gaps, but the larger gap in March 2010 would need two cycles/humps, which would not be filled well with a cubic approximation.

In [ ]:
df_i = fill_with_hard_limit(df, method="cubic", limit=25, columns=[TEMP], add_was_filled=True)
px.scatter(df_i, x=TIME, y=TEMP, color=TEMP + "_filled").update_traces(marker={'size': 3})

In [ ]:
to_ts(df_i[[TIME, TEMP]]).gaps().sort_values("gap_size", ascending=False)

In [ ]:
df[TEMP] = df_i[TEMP]
df.loc[df_i[TEMP+"_filled"], "filled"] = "cubic"

In [ ]:
px.scatter(df, x=TIME, y=TEMP, color="filled").update_traces(marker={'size': 3})

In [ ]:
df_i = fill_with_hard_limit(df, method="akima", limit=200, columns=[TEMP], add_was_filled=True)
px.scatter(df_i, x=TIME, y=TEMP, color=TEMP + "_filled").update_traces(marker={'size': 3})

Unfortunately, the large gaps are hard to fill with the methods of interpolate; none of them seem to capture patterns plausibly.



TODO Whether more interpolation should be done depends on the forecasting method to use. When training on multiple series isn't possible, you must fill _somehow_ anyway. If it is possible, you just lose some fixed amount of extra data per gap because the gap cannot be contained in the used lags, nor the predicted horizon (because of validation).

Could also use a simple model to interpolate, then use that "good enough" data for more complex modelling. The amount of non-trivially fillable missing data is small.

After some research, using PyPots and BRITS to impute dataset is probably best. Filling very small gaps with linear is still acceptable probably, but might as well try to do it entirely with BRITS and see how that performs. There are other models available, but it seems that BRITS would perform well on a dataset like this, given the benchmarking done by the PyPots devs.

Also, instead of getting carried away with imputation, you should start looking at seasonality etc.